huggingface/transformers

typical workflows  
initiate a model with model_type and model_name, e.g.,  
'distilbert', 'distilbert-base-uncased-distilled-squad' in https://www.kaggle.com/jonathanbesomi/question-answering-starter-pack  
'roberta', 'roberta-large' in https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing  
respectivly

More on  
https://huggingface.co/transformers/custom_datasets.html

transformers.roberta  
https://huggingface.co/transformers/model_doc/roberta.html  

Checking for the architecture  
roberta = transformers.RobertaModel.from_pretrained(ROBERTA_PATH)  
roberta

embedding  
encoder comprised of 12 RobertaLayer  
pooler of Dense with tanh activation

INPUT for roberta  
forward(self, input_ids, attention_mask, token_type_ids, ...)  
input_ids:  
Indices of input sequence tokens in the vocabulary.  
attention_mask:  
Mask to avoid performing attention on padding token indices.  
https://huggingface.co/transformers/glossary.html#attention-mask  
This can then be converted into a tensor in PyTorch or TensorFlow. The attention mask is a binary tensor indicating the position of the padded indices so that the model does not attend to them.  
token_type_ids:  
https://huggingface.co/transformers/glossary.html#token-type-ids  
Some models’ purpose is to do sequence classification or question answering. These require two different sequences to be joined in a single “input_ids” entry, which usually is performed with the help of special tokens, such as the classifier ([CLS]) and separator ([SEP]) tokens. For example, the BERT model builds its two sequence input as such:  
The first sequence, the “context” used for the question, has all its tokens represented by a 0, whereas the second sequence, corresponding to the “question”, has all its tokens represented by a 1.

In [ ]:
import tokenizers
tokernizers to tokenize, numericalize

In [ ]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab = f"{ROBERTA_PATH}/vocab.json",
    merges = f"{ROBERTA_PATH}/merges.txt",
    lowercase=True,
    add_prefix_space=True,
)

In [ ]:
tok = tokenizer.encode(STR)
tok.offstes : 
tok.ids : the dictionary index for the words in STR

***

simpletransformers

transformers vs simpletransformers  
simpletransformers is the package built on transformers for easy use  
model_type, model_name arguments are identical  
?the form required for train data is also identical  

Usual steps for using simpletransformrs  
Preparing train data as json of the format
https://simpletransformers.ai/docs/qa-data-formats/

Initiate a model with model_type and model_name  
model_type : bert, roberta, distilled, etc.  
model_name: specifies the exact architecture and trained weights to use  
https://simpletransformers.ai/docs/usage/#task-specific-models  
https://simpletransformers.ai/docs/qa-model/

model.train_model(PATH_TO_JSON)

***

spaCy

In [ ]:
HOW to use SPACY  
codes similar here  
https://yujuwon.tistory.com/entry/spaCy-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-Training-Models

https://towardsdatascience.com/custom-named-entity-recognition-using-spacy-7140ebbb3718
    
https://www.kaggle.com/rohitsingh9990/ner-training-using-spacy-ensemble?scriptVersionId=0

In [ ]:
training data for entity recognition  
(text, {'entities': [(start, end, class)]})  
text[start:end] -> entities in text  

Example  
('I like London and Berlin.', { 'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')] })

The belows are from https://www.kaggle.com/sblroid/tweet-sentiment-extraction-eda

In [ ]:
def get_training_data(sentiment):
    '''
    Returns Trainong data in the format needed to train spacy NER
    '''
    train_data = []
    for index, row in df_train.iterrows():
        if row.sentiment == sentiment:
            selected_text = row.selected_text
            text = row.text
            start = text.find(selected_text)
            end = start + len(selected_text)
            train_data.append((text, {"entities": [[start, end, 'selected_text']]}))
            # (text, selected_text)
            # selected_text = text[start:end]
    return train_data

In [ ]:
def get_model_out_path(sentiment):
    if sentiment == 'positive':
        model_out_path = 'models/model_pos'
    elif sentiment == 'negative':
        model_out_path = 'models/model_neg'
    else:
        model_out_path = None
    return model_out_path

In [ ]:
def train(train_data, output_dir, n_iter=20, model=None):
    if model is not None:
        nlp = spacy.load(output_dir)
        print(f"Loaded model '{model}'")
    else:
        nlp = spacy.blank('en')
        print("Created blank 'en' model")
        
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    else:
        ner = nlp.get_pipe('ner')
    # as a result, a pipe in nlp and we got ner to add labels
        
    for _, annotations in train_data:  # annotations : Dict
        for ent in annotations.get('entities'):  # 'entities': [[start, end, 'selected_text']]
            ner.add_label(ent[2])  # ent[2]: 'selected_text'
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()
            
        for itn in range(n_iter):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4, 500, 1.001))
            # minibatch : given train_data, returns a generator yielding batches of the given size
            # compounding(4, 500, 1.001):start 4, next 4 * 1.001, end 500
            # in this example, size=4 and size=compounding(4, 500, 1.001) generates the same result
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                # texts: a tuple of text of size 4 
                # annotations: a tuple of dict of size 4
                nlp.update(texts, annotations, drop=0.5, losses=losses)
            print('Losses', losses)
    save_model(output_dir, nlp, 'st_ner')